In [109]:
import cv2
import os
from custom_dset_new import train_val_test_split
import numpy as np
from tqdm import tqdm

In [129]:
data_dir = '/Users/ruoyangzhang/Documents/PythonWorkingDirectory/Assignment_data/images'

In [130]:
train_data, val_data, test_data = train_val_test_split(data_dir, train_split=0.8, val_split=0.2, test_split = 0)

In [131]:
img = cv2.imread(list(train_data.keys())[0])
b, g, r = cv2.split(img)
rgb_img = cv2.merge([r, g, b])

In [132]:
rgb_img.shape

(28, 28, 3)

In [133]:
ordered_train_dirs = [dir for dir in sorted(list(train_data.keys())) if os.path.split(dir)[-1] != '.DS_Store']

In [134]:
ordered_train_dirs[0]

'/Users/ruoyangzhang/Documents/PythonWorkingDirectory/Assignment_data/images/barren_land/0.png'

In [135]:
def convert_to_vector(img_dir):
    img = cv2.imread(img_dir)
    b, g, r = cv2.split(img)
    rgb_img = cv2.merge([r, g, b])
    rgb_img.shape = (1, 28*28*3)
    return(rgb_img)

In [136]:
input_images = np.array([convert_to_vector(dir) for dir in tqdm(ordered_train_dirs)])


 12%|█▏        | 31404/258590 [00:13<01:41, 2245.47it/s]


 25%|██▌       | 65143/258590 [00:27<01:23, 2330.34it/s]


 38%|███▊      | 98493/258590 [00:42<01:08, 2329.65it/s]


 50%|█████     | 129945/258590 [00:56<00:55, 2315.42it/s]


 63%|██████▎   | 161643/258590 [01:09<00:41, 2311.79it/s]


 75%|███████▍  | 192800/258590 [01:23<00:28, 2298.00it/s]


 86%|████████▋ | 223109/258590 [01:37<00:15, 2279.74it/s]


 97%|█████████▋| 249659/258590 [01:51<00:04, 2231.98it/s]


100%|██████████| 258590/258590 [01:56<00:00, 2213.14it/s]

In [138]:
input_images.shape = (input_images.shape[0], input_images.shape[2])

In [139]:
input_images

array([[177, 152, 129, ..., 182, 160, 136],
       [133, 137, 116, ..., 130, 139, 117],
       [158, 152, 125, ..., 123, 142, 111],
       ...,
       [ 49,  72,  93, ..., 100, 128, 117],
       [ 39,  55,  82, ...,  33,  58,  77],
       [104, 142, 124, ...,  65, 103, 106]], dtype=uint8)

In [140]:
labels = np.array([train_data[dir] for dir in ordered_train_dirs])

In [141]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [142]:
clf=RandomForestClassifier(n_estimators=100, n_jobs = 5)

In [143]:
clf.fit(input_images,labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=5,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Now we make test data into np.arrays

In [144]:
ordered_test_dirs = [dir for dir in sorted(list(val_data.keys())) if os.path.split(dir)[-1] != '.DS_Store']

In [145]:
test_images = np.array([convert_to_vector(dir) for dir in tqdm(ordered_test_dirs)])


 43%|████▎     | 27924/64646 [00:14<00:19, 1923.25it/s]


 87%|████████▋ | 56148/64646 [00:28<00:04, 1960.32it/s]


100%|██████████| 64646/64646 [00:32<00:00, 1977.49it/s]

In [146]:
test_images.shape

(64646, 1, 2352)

In [147]:
test_images

array([[[202, 187, 160, ..., 208, 194, 162]],

       [[171, 147, 125, ..., 123, 126, 108]],

       [[173, 158, 138, ..., 167, 162, 134]],

       ...,

       [[ 31,  53,  89, ...,  42,  50,  95]],

       [[108, 133, 156, ...,  90, 106, 127]],

       [[139, 125, 151, ...,  51,  61, 105]]], dtype=uint8)

In [148]:
test_images.shape = (test_images.shape[0], test_images.shape[2])

In [155]:
preds = clf.predict(test_images)

In [156]:
test_labels = np.array([val_data[dir] for dir in ordered_test_dirs])

In [157]:
from sklearn import metrics

In [158]:
print("Accuracy:",metrics.accuracy_score(test_labels, preds))

Accuracy: 0.9676391424063361


In [159]:
metrics.confusion_matrix(test_labels, preds)

array([[24101,     0,     0,     0,     0,     0],
       [   13, 11082,     0,     1,     0,    60],
       [   96,    37,  1286,    26,   124,    37],
       [    3,    55,     1, 14256,    17,   494],
       [   22,     0,    49,     0,  2928,     0],
       [   25,   591,     1,   439,     1,  8901]])

In [161]:
import xgboost as xgb

In [162]:
data_dmatrix = xgb.DMatrix(data = input_images, label = labels)

In [ ]:
xg_clf = xgb.XGBClassifier(objective='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 100, alpha = 10, n_estimators=2000)

In [ ]:
xg_clf.fit(input_images,labels)

preds = xg_reg.predict(test)